<a href="https://colab.research.google.com/github/dhruv369/ComputerVision/blob/master/stable_diffusion_xl_1_0_inpainting_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U diffusers

## Local Inference on GPU
Model page: https://huggingface.co/diffusers/stable-diffusion-xl-1.0-inpainting-0.1

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/diffusers/stable-diffusion-xl-1.0-inpainting-0.1)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
pip install -U diffusers transformers accelerate

In [ ]:
import torch
from diffusers import DiffusionPipeline

# switch to "mps" for apple devices
pipe = DiffusionPipeline.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", dtype=torch.bfloat16, device_map="cuda")

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipe(prompt).images[0]